<a href="https://colab.research.google.com/github/gmxavier/FAT0413150/blob/main/notebooks/05.03-Lunar-Lander-Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
try: 
  import control as ct
except:
  !pip install control
  import control as ct

# Lunar lander control

We will use all the codes we've done so far (e.g. models, simulation & plot functions). Everything we will need is [here](https://github.com/gmxavier/FAT0413150/blob/main/src/dynamic_models.py) and to load them we need just run the cell below.



In [192]:
# To get the file
from requests import get
repo_url = 'https://raw.githubusercontent.com/gmxavier/FAT0413150/main/'
for file in ['lunar_lander.py']:
  url = repo_url+'src/'+file
  with open(file, 'w') as f: 
    f.write(get(url).text)

# To load it and import the model update function
%load lunar_lander.py
from lunar_lander import lunar_engine_output, lunar_update, landing_plot

# To create the input/output model for the engine
lunar_engine = ct.NonlinearIOSystem(None, 
                                    lunar_engine_output, 
                                    inputs  = ('T_t', 'T_l', 'm_t'), 
                                    outputs = ('F_t', 'F_l'), 
                                    name    = 'lunar_engine')

# To create an input/output model for the dynamics
lunar_dynamics = ct.NonlinearIOSystem(lunar_update, 
                                      None, 
                                      inputs  = ('F_t', 'F_l'), 
                                      outputs = ('p_x', 'v_x', 'p_y', 'v_y', 'theta', 'v_theta', 'm_t', 'm_f'), 
                                      states  = ('p_x', 'v_x', 'p_y', 'v_y', 'theta', 'v_theta', 'm_t', 'm_f'), 
                                      name    = 'lunar_dynamics')

## PID control

Let's try the following D controller for the vertical speed $v_y$ (see that $K_P=0$ and $K_I=0$).

In [193]:
K_P = 0
K_I = 0
K_D = -1/25
p_y_P = ct.tf2io(ct.tf([K_P], [0, 1]), inputs='u', outputs='y', name='p_y_P')
p_y_I = ct.tf2io(ct.tf([K_I], [1, 0]), inputs='u', outputs='y', name='p_y_I')
p_y_D = ct.tf2io(ct.tf([K_D], [0, 1]), inputs='u', outputs='y', name='p_y_D')

Now let's build the closed loop control system, setup the simulation and see what happens.

In [194]:
# Closed loop control system
land = ct.InterconnectedSystem(
    # List of subsystems
    (lunar_engine, lunar_dynamics, p_y_P, p_y_I, p_y_D), name='land',

    # Interconnections between  subsystems
    connections=(
        ['p_y_P.u', 'lunar_dynamics.p_y'],
        ['p_y_I.u', 'lunar_dynamics.p_y'],
        ['p_y_D.u', 'lunar_dynamics.v_y'],
        ['lunar_engine.T_t', 'p_y_P.y', 'p_y_I.y', 'p_y_D.y'],
        ['lunar_engine.m_t', 'lunar_dynamics.m_t'],
        ['lunar_dynamics.F_t', 'lunar_engine.F_t'],
        ['lunar_dynamics.F_l', 'lunar_engine.F_l']
    ),

    # System inputs
    inplist=['lunar_engine.T_l'],
    inputs=['T_l'],

    #  System outputs
    outlist=['lunar_engine.T_t', 'lunar_engine.T_l', 'lunar_dynamics.p_x', 'lunar_dynamics.v_x', 
             'lunar_dynamics.p_y', 'lunar_dynamics.v_y', 'lunar_dynamics.theta', 'lunar_dynamics.v_theta', 
             'lunar_dynamics.m_t', 'lunar_dynamics.m_f'],
    outputs=['T_t', 'T_l', 'p_x', 'v_x', 'p_y', 'v_y', 'theta', 'v_theta', 'm_t', 'm_f']
)

In [197]:
# Initial conditions
X0 = [0.5e+03, 0.0, 160.0e+03, -0.7e+03, 0.0, 0.0, 15.0e+03, 8.0e+03]
# Total simulation time
tst = 1500
# Simulation time array
T  = np.linspace(0, tst, 10*tst+1)
# Input
T_l  = np.array([0.0 for t in T])
# Simulate and plot
res = landing_plot(land, T, T_l, X0)

NameError: ignored